In [1]:
from mdagent import MDAgent

In [2]:
#todays date and time
import datetime

start = datetime.datetime.now()
date = start.strftime("%Y-%m-%d")
print("date:",date)
time = start.strftime("%H:%M:%S")
print("time:",time)

date: 2024-07-22
time: 16:18:23


In [3]:
model = "gpt-4-1106-preview"
mda = MDAgent(
    agent_type='Structured', 
    model=model, 
    tools_model=model,
    use_memory=False, 
    top_k_tools="all"
)
print("LLM: ", mda.llm.model_name,"\nTemperature: ", mda.llm.temperature)

LLM:  gpt-4-1106-preview 
Temperature:  0.1


In [4]:
prompt = '''Simulate fibronectin (PDB ID 1FNF) for 1ns. Use an appropriate temperature from literature.'''
answer = mda.run(prompt)

Thought: To simulate the protein fibronectin with PDB ID 1FNF for 1 nanosecond, I need to find an appropriate temperature for the simulation from the literature. Once I have the temperature, I can use the SetUpandRunFunction tool to perform the simulation. First, I will use the LiteratureSearch tool to find the appropriate temperature for the simulation of fibronectin.

Action:
```
{
  "action": "LiteratureSearch",
  "action_input": {
    "query": "simulation temperature for fibronectin"
  }
}
```
"fibronectin simulation temperature study"
Search: "fibronectin simulation temperature study"


	Scraper openaccess failed on paper titled 'Degrees of freedom of atoms in a rigid molecule for local temperature calculation in molecular dynamics simulation'.
Traceback (most recent call last):
  File "/Users/jorgemedina/miniforge3/envs/mdagent2/lib/python3.11/site-packages/paperscraper/scraper.py", line 82, in scrape
    result = await scraper.function(paper, path, **scraper.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jorgemedina/miniforge3/envs/mdagent2/lib/python3.11/site-packages/paperscraper/lib.py", line 300, in openaccess_scraper
    await link_to_pdf(url, path, session)
  File "/Users/jorgemedina/miniforge3/envs/mdagent2/lib/python3.11/site-packages/paperscraper/lib.py", line 133, in link_to_pdf
    r.raise_for_status()
  File "/Users/jorgemedina/miniforge3/envs/mdagent2/lib/python3.11/site-packages/aiohttp/client_reqrep.py", line 1070, in raise_for_status
    raise ClientResponseError(
aiohttp.client_exceptions.ClientResponseErro


Found 12 papers
Since the LiteratureSearch tool did not provide an answer, I will use a standard temperature that is commonly used in molecular dynamics simulations of proteins, which is 300 Kelvin (approximately 27 degrees Celsius). This temperature is close to human body temperature and is often used as a standard in simulations unless specific literature suggests otherwise. With this temperature, I can proceed to set up and run the simulation for fibronectin (PDB ID 1FNF) for 1 nanosecond using the SetUpandRunFunction tool.

Action:
```
{
  "action": "SetUpandRunFunction",
  "action_input": {
    "pdb_id": "1FNF",
    "forcefield_files": ["amber99sb-ildn.xml", "tip3p.xml"],
    "save": true,
    "system_params": {
      "nonbondedMethod": "PME",
      "nonbondedCutoff": "1 * nanometers",
      "constraints": "HBonds",
      "rigidWater": true,
      "solvate": true
    },
    "integrator_params": {
      "integrator_type": "LangevinMiddle",
      "Temperature": "300 * kelvin",
    

In [5]:
end = datetime.datetime.now()
elapsed = end - start
print(f"duration: {elapsed.total_seconds()/60:.2f} minutes")

duration: 1.20 minutes


In [6]:
registry = mda.path_registry
all_names = registry.list_path_names()
print(all_names)

No names found. The JSON file is empty or does not contain name mappings.


In [7]:
# ensure all files are in path registry
assert all(n in all_names for n in ['1FNF','sim0', 'top_sim0', 'rec0']), "Not all file ids are present in path registry"

AssertionError: Not all file ids are present in path registry

In [ ]:
import mdtraj as md
import os
path_traj = registry.get_mapped_path("rec0_111311")
path_top = registry.get_mapped_path("top_sim0_111311")

assert os.path.exists(path_traj), f"Trajectory file not found: {path_traj}"
assert os.path.exists(path_top), f"Topology file not found: {path_top}"
traj = md.load(path_traj, top=path_top)
print(f"number of frames: {traj.n_frames}")
print(f"number of residues: {traj.n_residues}")

# Experiment Result:
### Completed without Exception or TimeOut Errors ✅
### Attempted all necessary steps ❌
### Completed without Hallucination ❌
### Logic make sense ❌
### Correct Answer  ❌